# 🎬 Vídeo → Desenho/Anime no Google Colab

Este notebook oferece **duas rotas**:

1. **Cartoon Rápido (OpenCV)** — efeito de bordas + cores chapadas (leve, CPU).
2. **AnimeGANv2** — estilo anime mais bonito (pode ser mais lento, usa IA).

## Como usar
1. Execute a célula **Setup Geral**.
2. Em **Upload do vídeo**, envie seu arquivo `.mp4`.
3. Escolha **Cartoon Rápido** _ou_ **AnimeGANv2** e execute as células da rota escolhida.
4. Baixe o resultado ao final.

> Dica: Se der erro ao baixar pesos do AnimeGANv2, reexecute a célula ou siga o link do repo para obter os pesos atualizados.


In [0]:
# === Setup Geral ===
!pip -q install moviepy==1.0.3 opencv-python==4.10.0.84
import os, subprocess, shlex
os.makedirs('work/frames_in', exist_ok=True)
os.makedirs('work/frames_out', exist_ok=True)
print('Ambiente pronto.')


In [0]:
# === Upload do vídeo ===
from google.colab import files
up = files.upload()
input_name = list(up.keys())[0]
print('Arquivo recebido:', input_name)

# Extrair áudio original (para recolocar depois)
audio_path = 'work/audio.aac'
subprocess.run(shlex.split(f'ffmpeg -y -i "{input_name}" -vn -acodec aac "{audio_path}"'))

# Extrair frames
subprocess.run(shlex.split(f'ffmpeg -y -i "{input_name}" work/frames_in/%06d.png'))
print('Frames extraídos em work/frames_in')


## 🟦 Rota 1 — Cartoon Rápido (OpenCV)
Aplica um filtro de bordas + cores chapadas em cada frame e recompõe o vídeo.


In [0]:
import cv2, numpy as np, glob

def cartoonize(img):
    color = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)
    gray  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray  = cv2.medianBlur(gray, 7)
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                  cv2.THRESH_BINARY, 9, 2)
    edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    cartoon = cv2.bitwise_and(color, edges)
    return cartoon

in_paths = sorted(glob.glob('work/frames_in/*.png'))
os.makedirs('work/frames_out', exist_ok=True)

for i, p in enumerate(in_paths):
    img = cv2.imread(p)
    out = cartoonize(img)
    cv2.imwrite(f'work/frames_out/{i:06d}.png', out)
    if i % 50 == 0: print('Processados:', i)
print('Cartoonização concluída.')


In [0]:
# Recriar vídeo + recolocar áudio
subprocess.run(shlex.split('ffmpeg -y -framerate 30 -i work/frames_out/%06d.png -c:v libx264 -pix_fmt yuv420p work/out_silent.mp4'))
subprocess.run(shlex.split('ffmpeg -y -i work/out_silent.mp4 -i work/audio.aac -c:v copy -c:a aac -shortest video_cartoon_opencv.mp4'))
print('Vídeo gerado: video_cartoon_opencv.mp4')


## 🟨 Rota 2 — AnimeGANv2 (Anime Look)
Usa um modelo pré-treinado para transformar os frames em estilo anime.

**Observações:**
- Baixar pesos pode falhar às vezes. Se falhar, consulte o README do repositório e reenvie os pesos para a pasta `weights/` (o nome do arquivo `.h5`).


In [0]:
# Instalar dependências e clonar o repositório
!pip -q install tensorflow==2.15.0 opencv-python
!git clone https://github.com/TachibanaYoshino/AnimeGANv2
%cd AnimeGANv2
os.makedirs('weights', exist_ok=True)

print('⚠️ Se o download dos pesos falhar, faça upload manual para ./weights/')
try:
    # Exemplo de peso público (pode sair do ar). Substitua pelo link válido do README se necessário.
    !pip -q install gdown
    import gdown
    gdown.download(id='1qR_8N6GK4PjzF5Kc-jsY0p1T3Ih7i9n_', output='weights/face_paint_512_v2.h5', quiet=True)
except Exception as e:
    print('Falha no download automático dos pesos:', e)

%cd /content


In [0]:
# Processar frames com AnimeGANv2
import glob, os
in_paths = sorted(glob.glob('work/frames_in/*.png'))
os.makedirs('work/frames_out', exist_ok=True)

# O repositório inclui utilitários. Usaremos um script simples baseado em OpenCV + modelo TF.
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2

weights_path = '/content/AnimeGANv2/weights/face_paint_512_v2.h5'
assert os.path.exists(weights_path), 'Peso não encontrado. Coloque o .h5 em AnimeGANv2/weights/'

model = load_model(weights_path, compile=False)

def infer_anime(img_bgr):
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    h, w = img_rgb.shape[:2]
    inp = cv2.resize(img_rgb, (512, 512))
    inp = inp.astype('float32')/255.0
    inp = inp[None, ...]
    out = model.predict(inp, verbose=0)[0]
    out = (out*255.0).clip(0,255).astype('uint8')
    out = cv2.resize(out, (w, h))
    out_bgr = cv2.cvtColor(out, cv2.COLOR_RGB2BGR)
    return out_bgr

for i, p in enumerate(in_paths):
    img = cv2.imread(p)
    out = infer_anime(img)
    cv2.imwrite(f'work/frames_out/{i:06d}.png', out)
    if i % 20 == 0: print('Frames AnimeGANv2:', i)
print('AnimeGANv2 concluído.')


In [0]:
# Recriar vídeo + recolocar áudio (AnimeGANv2)
subprocess.run(shlex.split('ffmpeg -y -framerate 30 -i work/frames_out/%06d.png -c:v libx264 -pix_fmt yuv420p work/out_silent.mp4'))
subprocess.run(shlex.split('ffmpeg -y -i work/out_silent.mp4 -i work/audio.aac -c:v copy -c:a aac -shortest video_cartoon_animeganv2.mp4'))
print('Vídeo gerado: video_cartoon_animeganv2.mp4')


## ⬇️ Download
Execute a célula abaixo para baixar o(s) arquivo(s) gerado(s).


In [0]:
from google.colab import files
for f in ['video_cartoon_opencv.mp4', 'video_cartoon_animeganv2.mp4']:
    if os.path.exists(f):
        files.download(f)
print('Concluído. Se não aparecer, use a aba Files à esquerda e clique com o botão direito > Download.')
